In [23]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# custom code for collecting data
from src.data.scraper import load_cached_etf_tickers
from src.data.data_collection import gather_data

# custom code for collecting cached data
from src.data.data_collection_cache import gather_data_cached, save_data, _tickers_to_hash, _get_filename

### Configs - change these to the desired values to cache as wanted
startDateStr = '2010-10-01'
endDateStr = '2024-10-02' # documentation said that endDateStr is exclusive for both yahoofinance and the original code, but actually printing the shapes showed otherwise..
instrumentIdsNASDAQandNYSE = load_cached_etf_tickers()
###
# live_data = gather_data(startDateStr, endDateStr, instrumentIdsNASDAQandNYSE)

In [13]:
save_data(live_data, startDateStr, endDateStr, instrumentIdsNASDAQandNYSE, cache_dir='../src/data/cache')
live_data

{'close': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'open': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'high': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'low': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'vol': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'yfinance_formatted': Empty DataFrame
 Columns: [(MEDX, Open), (MEDX, High), (MEDX, Low), (MEDX, Close), (MEDX, Adj Close), (MEDX, Volume), (DFGX, Open), (DFGX, High), (DFGX, Low), (DFGX, Close), (DFGX, Adj Close), (DFGX, Volume)]
 Index: []}

In [ ]:
cached_data = gather_data_cached(startDateStr, endDateStr, instrumentIdsNASDAQandNYSE, cache_dir='../src/data/cache')
assert len(live_data) == len(cached_data), "The lengths of live and cached data do not match."
cached_data

{'close': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'open': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'high': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'low': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'vol': Empty DataFrame
 Columns: [MEDX, DFGX]
 Index: [],
 'yfinance_formatted': Empty DataFrame
 Columns: [(MEDX, Open), (MEDX, High), (MEDX, Low), (MEDX, Close), (MEDX, Adj Close), (MEDX, Volume), (DFGX, Open), (DFGX, High), (DFGX, Low), (DFGX, Close), (DFGX, Adj Close), (DFGX, Volume)]
 Index: []}

In [ ]:
filename = _get_filename(startDateStr, endDateStr, instrumentIdsNASDAQandNYSE)
cached_data = gather_data_cached(startDateStr, endDateStr, instrumentIdsNASDAQandNYSE, cache_dir='../src/data/cache')